In [4]:
import requests
import pandas as pd

In [19]:
def get_top_coins():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 10,
        "page": 1
    }

    response = requests.get(url, params=params)
    data = response.json()

    return data

top_coins = get_top_coins()

for index, coin in enumerate(top_coins, start=1):
    print(f"{index}. {coin['name']} ({coin['symbol']}): ${coin['current_price']}")

1. Bitcoin (btc): $26543
2. Ethereum (eth): $1682.34
3. Tether (usdt): $1.001
4. BNB (bnb): $216.05
5. XRP (xrp): $0.493757
6. USD Coin (usdc): $0.998545
7. Lido Staked Ether (steth): $1680.85
8. Cardano (ada): $0.257069
9. Solana (sol): $21.59
10. Dogecoin (doge): $0.060487


In [12]:
def get_historical_data():
    url = f"https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"

    params = {
        'vs_currency': 'usd',
        'days': '2160',
        'interval': 'daily'
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Error al obtener los datos:", response.status_code)
        return None


In [13]:
historical_data = get_historical_data()

df = pd.DataFrame(historical_data)
df.head()

,prices,market_caps,total_volumes
0,"[1505779200000, 3898.93473248234]","[1505779200000, 64629619385.94208]","[1505779200000, 942656241.4373351]"
1,"[1505865600000, 3858.02095060051]","[1505865600000, 63957210363.29384]","[1505865600000, 676074373.1781092]"
2,"[1505952000000, 3628.05858753461]","[1505952000000, 60152757874.00039]","[1505952000000, 913652257.307351]"
3,"[1506038400000, 3622.1830393146]","[1506038400000, 60061814860.04741]","[1506038400000, 767837732.7568976]"
4,"[1506124800000, 3763.41140627887]","[1506124800000, 62411285738.3069]","[1506124800000, 578068211.7796992]"


In [14]:
df['market_caps'] = df['market_caps'].apply(lambda x: int(x[1]))
df['total_volumes'] = df['total_volumes'].apply(lambda x: int(x[1]))
df['Date'] = df['prices'].apply(lambda x: pd.to_datetime(x[0], unit='ms'))
df['prices'] = df['prices'].apply(lambda x: int(x[1]))

In [15]:
df.head()

,prices,market_caps,total_volumes,Date
0,3898,64629619385,942656241,2017-09-19
1,3858,63957210363,676074373,2017-09-20
2,3628,60152757874,913652257,2017-09-21
3,3622,60061814860,767837732,2017-09-22
4,3763,62411285738,578068211,2017-09-23


In [16]:
df.rename(columns={'prices': 'Prices_usd','total_volumes':'Volume','market_caps':'Market_caps'}, inplace=True)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2161 entries, 0 to 2160
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Prices_usd   2161 non-null   int64         
 1   Market_caps  2161 non-null   int64         
 2   Volume       2161 non-null   int64         
 3   Date         2161 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(3)
memory usage: 67.7 KB


In [13]:
df.to_csv('./Data/BTCUSD.csv')